# Czyszczenie i analiza danych

Celem naszego zadania jest przygotowanie zestawu danych zawartego w pliku TitanicMess.tsv, oczyszczenie go, a następnie analiza. Zestaw zawiera informacje na temat podróżujących statkiem Titanic.


In [24]:
# Lista zaimportowanych bibliotek do przygotowania danych

import pandas as pandas
from pandas.api.types import is_numeric_dtype
import re
import math
from statistics import mean

## Wczytanie zestawu danych

In [25]:
# wczytanie zbioru i wyświetlenie jego pierwszych rekordów

dataSet = pandas.read_csv("TitanicMess.tsv", sep="\t")
dataSet.head(15)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S,Titanic
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C,Titanic
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S,Titanic
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S,Titanic
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S,Titanic
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,"8,4583",NaN,Q,Titanic
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,"51,8625",E46,S,Titanic
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,"21,075",NaN,S,Titanic
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,"11,1333",NaN,S,Titanic
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,"30,0708",NaN,C,Titanic


## Zidentyfikowane problemy w trakcie przeglądania danych

### 1. Zbędne kolumny
Możemy zauważyć, że część kolumn jest nieistotna dla naszego zbioru danych z perspektywy analizy. Kolumna ship zawiera jedynie wartość Titanic, która jest oczywista dla zbioru danych.

In [26]:
dataSet['ship'].unique()

array(['Titanic'], dtype=object)

### 2. Powtarzające się wiersze tabeli
Możemy zauważyć powtarzający się wiersz z id = 11. Należy usunąć powielone wiersze, które zakłamują występowanie pewnego przypadku. Aby wyświetlić wszystkie powtórzenia, możemy wykorzystać funkcję `duplicated()` z biblioteki `pandas`. W poszukiwaniu powtórzonych elementów skupimy się na id, które powinno być unikalną wartością.

In [27]:
dataSet[dataSet.duplicated(['PassengerId'], keep=False)]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,ship
10,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
13,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
23,11,1,3,"Sandstrom, Miss. Marguerite Ru&5$$",female,4,1,1,PP 9549,"16,7",G6,S,Titanic
224,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
520,225,1,1,"Hoyt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic
678,225,1,1,"Hoytt, Mr. Frederick Maxfield",male,38,1,0,19943,90,C93,S,Titanic


### 3. Błędne wartości
Następnym krokiem będzie sprawdzenie poprawności danych, które zawierają się w poszczególnych kolumnach.

#### 3.1 Wartości numeryczne

In [28]:
[is_numeric_dtype(dataSet['PassengerId']),
 is_numeric_dtype(dataSet['Survived']),
 is_numeric_dtype(dataSet['Pclass']),
 is_numeric_dtype(dataSet['Age']),
 is_numeric_dtype(dataSet['SibSp']),
 is_numeric_dtype(dataSet['Parch']),
 is_numeric_dtype(dataSet['Fare'])]

[True, True, True, False, True, True, False]

Można zauważyć, że dla wieku oraz opłat za bilet nie posiadamy jedynie wartości numerycznych. Warto zatem sprawdzić te wartości i zamienić na poprawny format. Na początek sprawdźmy wiek. Jego wartości powinny być liczbami naturalnymi.

In [29]:
for age in dataSet['Age'].dropna():
    if not age.isdigit():
        print(age)

.9
28,5
0,83
14,5
70,5
32,5
32,5
36,5
55,5
40,5
45,5
20,5
23,5
0,92
45,5
0,75
-3
-12
40,5
0,75
24,5
28,5
0,67
30,5
0,42
30,5
0,83
34,5


Okazuje się, że istnieje wiele wartości z wartościami po przecinku. Istnieją również liczby ujemne, a także przypadek, który zaczyna się od kropki. Wartości te powinny zostać zastąpione liczbami naturalnymi.

Przejdźmy zatem do wartości kolumny z cenami biletów. Wartości powinny być tam zmiennoprzecinkowymi liczbami nieujemnymi.


In [30]:
for fare in dataSet['Fare'].dropna():
    if not fare.replace(',','',1).isnumeric():
        print(fare)

-90
15,9a


Możemy zauważyć, że w kolumnie znajduje się liczba ujemna oraz wartość zawierająca literę. Należy je zamienić wartościami zmiennoprzecinkowymi nieujemnymi. Dodatkowo można też zauważyć, że pozostałe wartości tej kolumny często zawierają więcej niż dwie cyfry po przecinku, co nie jest możliwe dla wartości pieniężnych. Należałoby zaokrąglić je do dwóch miejsc po przecinku.

### 3.2 Brakujące wartości
Przy pomocy biblioteki `pandas` możemy znaleźć wartości brakujące lub określone wyrażeniem `NaN`. Posłuży nam do tego funkcja `isna()`.


In [31]:
dataSet.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            173
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          685
Embarked         2
ship             0
dtype: int64

Widzimy zatem, że kolumny `Age`, `Cabin` oraz `Embarked` zawierają brakujące wartości.

### 3.3 Wartości odstające
Wśród opisywanych kolumn jedynie kolumna z wiekiem mogłaby zawierać wartości, które byłyby zbyt duże. Wiemy już, że pojawiają się wartości ujemne, natomiast musimy usunąć również wartości zbyt duże. Przyjmijmy, że niemożliwe do osiągnięcia jest posiadanie więcej niż 115 lat.

In [32]:
# Należy dodać lstrip(), ponieważ jeden wynik zaczyna się od kropki
for age in dataSet['Age'].dropna():
    if float(age.lstrip('.').replace(',','.',1)) > 115:
        print(age)

4435
250


Uzyskaliśmy zatem dwa wyniki, które nie są dla człowieka osiągalne w kwestii wieku. Należy zastanowić się, w jaki sposób trzeba je zamienić.

### 3.4 Znaki niepożądane

Już pierwsze wiersze całego zbioru danych pokazują, że wśród danych osobowych pasażerów mogą znajdować się znaki niealfanumeryczne.

In [33]:
for name in dataSet['Name']:
    x = re.match(r'.*[^A-Za-z\s,.\"\'\-()/].*', name)
    if x:
        print(name)


Sandstrom, Miss. Marguerite Ru&5$$
Sandstrom, Miss. Marguerite Ru&5$$
Sandstrom, Miss. Marguerite Ru&5$$


Udało nam się pokazać, że tylko jedno imię zawiera nieodpowiednie znaki.

### 3.5 Literówki
W swoich rozważaniach skupmy się na elementach, które nie zawierają zbyt wielu unikalnych wartości. Jeżeli jest ich zbyt dużo, trudno byłoby znaleźć w każdym możliwe zamiany liter.

In [34]:
dataSet.nunique() 

PassengerId    888
Survived         2
Pclass           3
Name           889
Sex              6
Age             93
SibSp            7
Parch            6
Ticket         680
Fare           250
Cabin          145
Embarked         6
ship             1
dtype: int64

Spośród powyższych wartości możemy odrzucić `Survived` oraz `Pclass`. Pierwsza kolumna zawiera jedynie jedynki oraz a druga jedną z trzech dostępnych klas. Widzimy zatem, że nie pojawiają się tam żadne nieoczekiwane wartości. Warto przyjrzeć się na pewno kolumnie `Sex`, która prawdopodobnie zawiera zbyt wiele opcji. Sytuacja ma się podobnie z trzema innymi kolumnami. Warto zatem przejrzeć możliwe wartości:

In [35]:
dataSet['Sex'].unique()

array(['male', 'female', 'malef', 'mal', 'fem', 'femmale'], dtype=object)

Możemy zauważyć, że w kolumnie `Sex` pojawiają się literówki. Naszym zadaniem będzie zatem podmiana błędnych wyrazów.

In [36]:
dataSet['SibSp'].unique()


array([1, 0, 3, 4, 2, 5, 8], dtype=int64)

Powyższa kolumna zawiera liczbę rodzeństwa/małżonków, którzy znajdują się również na pokładzie statku. Nie widać tutaj nieprawidłowych wartości.

In [37]:
dataSet['Parch'].unique()

array([0, 1, 2, 5, 3, 4], dtype=int64)

Kolejna kolumna zawiera informacje o liczbie rodziców/dzieci danego pasażera znajdujących się na pokładzie. Również nie widać tutaj niestandardowych wartości.

In [38]:
dataSet['Embarked'].unique()

array(['S', 'C', 'Q', 'So', nan, 'Co', 'Qe'], dtype=object)

Z informacji o zbiorze danych dowiadujemy się, że są jedynie 3 dostępne porty (C = Cherbourg, Q = Queenstown, S = Southampton). Część z wartości zawiera jednak po jednej dodatkowej literze, co jedynie wprowadza zamieszanie i należy je ujednolicić. Występują również braki danych w dwóch wierszach. Zaobserwowaliśmy to już poprzednich analizach.


## Czyszczenie zbioru danych

### 1. Zbędne kolumny

Pozbycie się jednej z kolumn jest bardzo szybką operacją. W celu potwierdzenia sprawdzimy pierwsze wiersze zbioru.

In [39]:
del dataSet['ship']
dataSet.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,"7,25",NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,"71,2833",C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,"7,925",NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,"53,1",C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,"8,05",NaN,S


### 2. Powtarzające się wiersze tabeli

Należy usunąć powielone wiersze.

In [40]:
dataSet = dataSet.drop_duplicates('PassengerId')
dataSet['PassengerId'].is_unique

True

Powyższa informacja daje nam pewność, że w zbiorze nie mamy już duplikacji.

### 3. Błędne wartości

#### 3.1 Wartości numeryczne
W kolumnie `Age` mieliśmy kilka przypadków niepoprawnych wartości. Jedną z nich była liczba rozpoczynająca się od kropki. Wydaje się, że wystarczy ją usunąć, aby wartość miała sens dla zbioru. Dodatkowo usunąć należy znak minusa przed liczbami ujemnymi oraz zaokrąglić wiek do pełnych lat. W tym celu użyje funkcji `ceil()`, aby nie uzyskać wartości 0.

In [41]:
for age in dataSet['Age'].dropna():
    if not age.isdigit():
        if age.startswith('.'):
            dataSet['Age'] = dataSet['Age'].replace(age, age.replace('.',''))
        if age.startswith('-'):
            dataSet['Age'] = dataSet['Age'].replace(age, age.replace('-',''))
        if age.find(',') != -1:
            dataSet['Age'] = dataSet['Age'].replace(age, str(math.ceil(float(age.replace(',','.')))))


for age in dataSet['Age'].dropna():
    if not age.isdigit():
        print(age)

Obecnie wszystkie wartości wieku są liczbami naturalnymi.

Dla kolumny `Fare` należy zamienić wartości ujemne oraz wyrazy zawierające znaki alfabetu, a dodatkowo zaokrąglić ceny biletów do dwóch miejsc po przecinku.

In [42]:
for fare in dataSet['Fare'].dropna():
    if fare.startswith('-'):
        dataSet['Fare'] = dataSet['Fare'].replace(fare, fare.replace('-',''))
    x = re.match(r'.*[A-Za-z].*', fare)
    if x:
        new = re.sub(r'[A-Za-z]', "", fare)
        dataSet['Fare'] = dataSet['Fare'].replace(fare, new)
        fare = new
    with_coma = float(fare.replace(',','.'))
    dataSet['Fare'] = dataSet['Fare'].replace(fare, str(round(with_coma, 2)))


for fare in dataSet['Fare'].dropna():
    if not fare.replace('.','',1).isnumeric():
        print(fare)

dataSet.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.46,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.86,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2,3,1,349909,21.07,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27,0,2,347742,11.13,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14,1,0,237736,30.07,NaN,C


Otrzymane wartości zgadzają się z naszymi założeniami.

#### 3.2 Brakujące wartości

Kolejnym napotkanym problemem były brakujące wartości w 3 kolumnach. Aby nie uzupełniać wieku zerami warto użyć średniej spośród pozostałych wartości. W przypadku portu startowego mamy jedynie dwa puste pola. Na potrzeby zadania załóżmy, że obie osoby wyruszyły z Southampton, które było portem startowym rejsu. Sprawa z kabinami jest znacznie trudniejsza do rozwiązania. Mamy tam bowiem sporo brakujących wartości i różne nazwy kabin. Ustalmy zatem słowo `unspecified`, które zastąpi puste pola.

In [43]:
do_not_count = 0
counter = 0
for age in dataSet['Age'].dropna():
    if int(age) > 115:    #Pętla na potrzeby kolejnego podpunktu. Nie chcemy, aby niepoprawne wyniki zawyżyły średnią.
        do_not_count += 1
        continue
    counter += int(age)
value = str(int(counter/(len(dataSet['Age'].dropna())-do_not_count)))

dataSet['Age'] = dataSet['Age'].fillna(value)
dataSet['Cabin'] = dataSet['Cabin'].fillna('unspecified')
dataSet['Embarked'] = dataSet['Embarked'].fillna('S')

dataSet.isna().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Powyższa lista przedstawia nam informację, że w żadnej kolumnie nie posiadamy już brakującej wartości.

#### 3.3 Wartości odstające

Na drodze analizy ustalono, że pewne wartości znajdujące się w kolumnie `Age` wykraczają poza możliwe wartości. Mowa tutaj o wieku pasażerów, który wynosił odpowiednio 4435 i 250 lat. W procesie czyszczenia danych zamienimy je na wartości średniej wieku pozostałych pasażerów.

In [44]:
for age in dataSet['Age']:
    if int(age) > 115:
        dataSet['Age'] = dataSet['Age'].replace(age, value)

max_value = 0
for age in dataSet['Age']:
    if int(age) > max_value:
        max_value = int(age)
print("Maksymalna wartość: ")
max_value

Maksymalna wartość: 


80

Widzimy, że obecnie najstarsza osoba ma 80 lat.

#### 3.4 Znaki niepożądane

W jednym z imion pojawiły się znaki specjalne, które nie są literami, ani separatorami pomiędzy częściami danych osobowych. W związku z tym postaramy się je usunąć, aby dane te były czytelne dla człowieka.

In [45]:
for name in dataSet['Name']:
    x = re.match(r'.*[^A-Za-z\s,.\"\'\-()/].*', name)
    if x:
        new_name = re.sub('[^A-Za-z\s,.\"\'\-()/]', '', name)
        dataSet['Name'] = dataSet['Name'].replace(name, new_name)
        print(new_name)

for name in dataSet['Name']:
    x = re.match(r'.*[^A-Za-z\s,.\"\'\-()/].*', name)
    if x:
        print(name)

Sandstrom, Miss. Marguerite Ru


Funkcja usunęła niepożądane znaki, dzięki czemu imię i nazwisko stało się bardziej czytelne.

#### 3.5 Literówki

Spośród przeglądanych kolumn okazało się, że błędy w pisowni lub oznaczeniach pojawiają się w sekcji `Sex` oraz `Embarked`. W tym celu postaramy się poprawić wyrazy. Dla płci pojawiły się drobne literówki, które łatwo zidentyfikować i zamienić na poprawną formę. W przypadku portów część z nich zawiera niepotrzebnie dodatkowe litery, które zostaną usunięte.

In [46]:
for sex in dataSet['Sex']:
    if not re.match('^female$|^male$', sex):
        if re.match('.*fem.*', sex):
            dataSet['Sex'] = dataSet['Sex'].replace(sex, 'female')
        else:
            dataSet['Sex'] = dataSet['Sex'].replace(sex, 'male')

for port in dataSet['Embarked']:
    if len(port) > 1:
        dataSet['Embarked'] = dataSet['Embarked'].replace(port, port[0])

print(dataSet['Sex'].unique())
print(dataSet['Embarked'].unique())

['male' 'female']
['S' 'C' 'Q']


Pozostały już jedynie unikalne wartości, które jasno opisują dane rekordy.


## Podsumowanie

W trakcie przeprowadzonej udało się zidentyfikować problemy, które zawarte zostały w 3 głównych punktach. Ostatni z nich zawierał 5 mniejszych podpunktów, które były ze sobą powiązane tematycznie. Wykryto m.in.: zbędne kolumny, powtarzające się wiersze, błędy w reprezentacji wartości liczbowych, brakujące lub odstające wartości, niepożądane znaki graficzne oraz błędy w pisowni wartości, które występowały w małej liczbie na przestrzeni całych kolumn.  

Udało się wyczyścić zbiór danych ze wszystkich z wymienionych problemów i dzięki temu stał się on łatwiejszy do odczytu, przetwarzania i dalszej analizy. Zbiór wynikowy zostanie zapisany w nowoutworzonym pliku:

In [47]:
dataSet.to_csv('TitanicCleaned.tsv', sep='\t')


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,unspecified,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.28,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.92,unspecified,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,unspecified,S
...,...,...,...,...,...,...,...,...,...,...,...,...
97,98,1,1,"Greenfield, Mr. William Bertram",male,23,0,1,PC 17759,63.36,D10 D12,C
98,99,1,2,"Doling, Mrs. John T (Ada Julia Bone)",female,34,0,1,231919,23.0,unspecified,S
99,100,0,2,"Kantor, Mr. Sinai",male,34,1,0,244367,26.0,unspecified,S
100,101,0,3,"Petranec, Miss. Matilda",female,28,0,0,349245,7.9,unspecified,S
